In [2]:
!pip install BeautifulSoup4
!pip install lxml
!pip install requests
!pip install geopy
import  requests
import json

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors

from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

     |████████████████████████████████| 122kB 7.4MB/s eta 0:00:01
     |████████████████████████████████| 5.5MB 8.3MB/s eta 0:00:01
     |████████████████████████████████| 112kB 5.7MB/s eta 0:00:01


In [3]:
#read table from Wiki page into pandas dataframe
df=pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
print(df.columns)
df.shape
df

Index(['Postal Code', 'Borough', 'Neighbourhood'], dtype='object')


,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [4]:
#remove cells with 'Not assigned' as borough and name df_filtered
df_filtered=df[df['Borough']!='Not assigned']

#checking if any duplicate postal codes exist by comparing length of filtered dataframe with length of unique postal codes
df_filtered['Postal Code'].value_counts()



M4G    1
M4M    1
M1L    1
M1W    1
M1K    1
      ..
M2L    1
M6H    1
M6N    1
M3L    1
M9A    1
Name: Postal Code, Length: 103, dtype: int64

In [5]:
#print shape of filtered dataframe
df_filtered.shape
df_filtered.reset_index

<bound method DataFrame.reset_index of     Postal Code           Borough  \
2           M3A        North York   
3           M4A        North York   
4           M5A  Downtown Toronto   
5           M6A        North York   
6           M7A  Downtown Toronto   
..          ...               ...   
160         M8X         Etobicoke   
165         M4Y  Downtown Toronto   
168         M7Y      East Toronto   
169         M8Y         Etobicoke   
178         M8Z         Etobicoke   

                                         Neighbourhood  
2                                            Parkwoods  
3                                     Victoria Village  
4                            Regent Park, Harbourfront  
5                     Lawrence Manor, Lawrence Heights  
6          Queen's Park, Ontario Provincial Government  
..                                                 ...  
160      The Kingsway, Montgomery Road, Old Mill North  
165                               Church and Wellesley  
168

In [6]:
#create df for coordinates from csv file
coordinates=pd.read_csv('http://cocl.us/Geospatial_data')
coordinates


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [112]:
#Merge 2 tables based on common column which is Postal Code
df_merged=pd.merge(df_filtered,coordinates, on='Postal Code')
df_merged

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


In [9]:
#Import required libraries
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')


Libraries imported.


In [10]:
#create map of toronto
map_toronto=folium.Map(location=[43.6532,-79.3832], zoom_start=10)

#add markers to map
for lat,lng,borough,neighbourood in zip(df_merged['Latitude'], df_merged['Longitude'], df_merged['Borough'],df_merged['Neighbourhood']):
    label='{},{}'.format('Neighbourhood','Borough') 
    label=folium.Popup(label,parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [11]:
LIMIT=100
radius=500
CLIENT_ID = 'EAICA3TLKIORL5UGMYU33N1BIKSKLUZKVG0UQ0WBZESP25I5' # your Foursquare ID
CLIENT_SECRET = 'IOAMSYO355V3S12Q0KZDFS4H3ZQDDQABM4HJZX2YOJXT5YXL' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


Your credentails:
CLIENT_ID: EAICA3TLKIORL5UGMYU33N1BIKSKLUZKVG0UQ0WBZESP25I5
CLIENT_SECRET:IOAMSYO355V3S12Q0KZDFS4H3ZQDDQABM4HJZX2YOJXT5YXL


In [14]:
# Explore the first Neighborhood in our dataframe.
# Get the Neighborhood's 
df_merged.loc[0, 'Neighbourhood']

'Parkwoods'

In [15]:
neighbourhood_latitude = df_merged.loc[0, 'Latitude'] # neighbourhood's latitude value
neighbourhood_longitude = df_merged.loc[0, 'Longitude'] # neighbourhood's longitude value

neighbourhood_name = df_merged.loc[0, 'Neighbourhood'] # neighbourhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of Parkwoods are 43.7532586, -79.3296565.


In [16]:
LIMIT = 100 # limit of number of venues returned by Foursquare API



radius = 500 # define radius


url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=EAICA3TLKIORL5UGMYU33N1BIKSKLUZKVG0UQ0WBZESP25I5&client_secret=IOAMSYO355V3S12Q0KZDFS4H3ZQDDQABM4HJZX2YOJXT5YXL&v=20180604&ll=43.7532586,-79.3296565&radius=500&limit=100'

In [19]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f27b31d780c337071eb6196'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Parkwoods - Donalda',
  'headerFullLocation': 'Parkwoods - Donalda, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 2,
  'suggestedBounds': {'ne': {'lat': 43.757758604500005,
    'lng': -79.32343823984928},
   'sw': {'lat': 43.7487585955, 'lng': -79.33587476015072}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e8d9dcdd5fbbbb6b3003c7b',
       'name': 'Brookbanks Park',
       'location': {'address': 'Toronto',
        'lat': 43.751976046055574,
        'lng': -79.33214044722958,
        'labeledLatLngs': 

In [22]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [24]:
from pandas.io.json import json_normalize
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  after removing the cwd from sys.path.


,name,categories,lat,lng
0,Brookbanks Park,Park,43.751976,-79.332140
1,Variety Store,Food & Drink Shop,43.751974,-79.333114


In [28]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return(nearby_venues)

In [31]:
toronto_venues = getNearbyVenues(names=df_merged['Neighbourhood'],
                                   latitudes=df_merged['Latitude'],
                                   longitudes=df_merged['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

In [32]:

toronto_venues.head()

,Neighbourhood,Neighbourhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


In [33]:
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Agincourt,4,4,4,4,4,4
"Alderwood, Long Branch",6,6,6,6,6,6
"Bathurst Manor, Wilson Heights, Downsview North",22,22,22,22,22,22
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",24,24,24,24,24,24
...,...,...,...,...,...,...
"Willowdale, Willowdale East",33,33,33,33,33,33
"Willowdale, Willowdale West",5,5,5,5,5,5
Woburn,4,4,4,4,4,4


In [39]:
#one hot coding to group each category and show occurence of each category
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")


toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

t_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
t_grouped

,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.041667,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,"Willowdale, Willowdale East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.030303,0.0,0.0,0.0,0.0,0.0,0.0
92,"Willowdale, Willowdale West",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
93,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
94,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [40]:

num_top_venues = 5

for hood in t_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = t_grouped[t_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0  Latin American Restaurant  0.25
1             Breakfast Spot  0.25
2                     Lounge  0.25
3               Skating Rink  0.25
4  Middle Eastern Restaurant  0.00


----Alderwood, Long Branch----
            venue  freq
0     Pizza Place  0.33
1             Gym  0.17
2  Sandwich Place  0.17
3     Coffee Shop  0.17
4             Pub  0.17


----Bathurst Manor, Wilson Heights, Downsview North----
                venue  freq
0                Bank  0.09
1         Coffee Shop  0.09
2      Sandwich Place  0.05
3       Deli / Bodega  0.05
4  Frozen Yogurt Shop  0.05


----Bayview Village----
                 venue  freq
0  Japanese Restaurant  0.25
1                 Café  0.25
2                 Bank  0.25
3   Chinese Restaurant  0.25
4        Movie Theater  0.00


----Bedford Park, Lawrence Manor East----
                  venue  freq
0    Italian Restaurant  0.08
1        Sandwich Place  0.08
2           Coffee Shop  0.08
3    

In [41]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [124]:
#To show top 10 venues for each neighbourhood to allow comparison:

num_top_venues = 10
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.Aappend('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{} Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = t_grouped['Neighbourhood']

for ind in np.arange(t_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(t_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1 Most Common Venue,2 Most Common Venue,3 Most Common Venue,4 Most Common Venue,5 Most Common Venue,6 Most Common Venue,7 Most Common Venue,8 Most Common Venue,9 Most Common Venue,10 Most Common Venue
0,Agincourt,Breakfast Spot,Latin American Restaurant,Skating Rink,Lounge,Yoga Studio,Dog Run,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
1,"Alderwood, Long Branch",Pizza Place,Gym,Coffee Shop,Sandwich Place,Pub,Discount Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner
2,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Ice Cream Shop,Pizza Place,Restaurant,Deli / Bodega,Chinese Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Middle Eastern Restaurant
3,Bayview Village,Café,Bank,Japanese Restaurant,Chinese Restaurant,Distribution Center,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Department Store
4,"Bedford Park, Lawrence Manor East",Sandwich Place,Coffee Shop,Restaurant,Italian Restaurant,Pizza Place,Pharmacy,Café,Butcher,Pub,Sushi Restaurant


In [48]:
neighbourhoods_venues_sorted.shape

(96, 11)

In [111]:
neighbourhoods_filtered=neighbourhoods_venues_sorted.loc[neighbourhoods_venues_sorted['1 Most Common Venue'].isin(['Pizza Place','Italian','Gym'])]
neighbourhoods_filtered

,Neighbourhood,1 Most Common Venue,2 Most Common Venue,3 Most Common Venue,4 Most Common Venue,5 Most Common Venue,6 Most Common Venue,7 Most Common Venue,8 Most Common Venue,9 Most Common Venue,10 Most Common Venue
1,"Alderwood, Long Branch",Pizza Place,Gym,Coffee Shop,Sandwich Place,Pub,Discount Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner
16,"Clarks Corners, Tam O'Shanter, Sullivan",Pizza Place,Fast Food Restaurant,Gas Station,Italian Restaurant,Rental Car Location,Fried Chicken Joint,Thai Restaurant,Chinese Restaurant,Convenience Store,Bank
22,Don Mills,Gym,Coffee Shop,Beer Store,Restaurant,Japanese Restaurant,Art Gallery,Chinese Restaurant,Sporting Goods Shop,Asian Restaurant,Clothing Store
38,Humber Summit,Gym,Pizza Place,Distribution Center,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
60,"Parkview Hill, Woodbine Gardens",Pizza Place,Gym / Fitness Center,Athletics & Sports,Gastropub,Intersection,Pharmacy,Café,Breakfast Spot,Bank,Pet Store
70,Scarborough Village,Pizza Place,Playground,Yoga Studio,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doner Restaurant
73,"St. James Town, Cabbagetown",Pizza Place,Coffee Shop,Bakery,Pub,Park,Italian Restaurant,Chinese Restaurant,Restaurant,Café,Butcher
87,Westmount,Pizza Place,Intersection,Coffee Shop,Sandwich Place,Middle Eastern Restaurant,Chinese Restaurant,Discount Store,Yoga Studio,Diner,Dessert Shop


In [57]:
neighbourhoods_filtered.shape

(8, 11)

There are 8 Neighbourhoods in Toronto where Italian food or Gyms are rated most common venues

Looking at the dataframe, Alderwood, Long Branch, Humber Summit, Parkview Hill and Woodbine Gardens have either Gym or Pizza as first and second option.
I will filter these into a new dataframe and map the neighbourhoods

In [61]:
chosen_neighbourhoods=neighbourhoods_venues_sorted.loc[neighbourhoods_venues_sorted['Neighbourhood'].isin(['Alderwood, Long Branch','Humber Summit','Parkview Hill, Woodbine Gardens'])]
chosen_neighbourhoods

,Neighbourhood,1 Most Common Venue,2 Most Common Venue,3 Most Common Venue,4 Most Common Venue,5 Most Common Venue,6 Most Common Venue,7 Most Common Venue,8 Most Common Venue,9 Most Common Venue,10 Most Common Venue
1,"Alderwood, Long Branch",Pizza Place,Gym,Coffee Shop,Sandwich Place,Pub,Discount Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner
38,Humber Summit,Gym,Pizza Place,Distribution Center,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
60,"Parkview Hill, Woodbine Gardens",Pizza Place,Gym / Fitness Center,Athletics & Sports,Gastropub,Intersection,Pharmacy,Café,Breakfast Spot,Bank,Pet Store


In [70]:
#original dataframe with coordinates of neighbourhoods
df_merged.head()
df_merged

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


In [77]:
#Adding coordinates of each chosen neighbourhood
df_merged1=df_merged.loc[df_merged['Neighbourhood'].isin(['Alderwood, Long Branch','Humber Summit','Parkview Hill, Woodbine Gardens'])]
df_merged1

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
50,M9L,North York,Humber Summit,43.756303,-79.565963
93,M8W,Etobicoke,"Alderwood, Long Branch",43.602414,-79.543484


In [79]:
map_toronto1=folium.Map(location=[43.6532,-79.3832], zoom_start=10)
#add markers to map
for lat,lng,borough,neighbourood in zip(df_merged1['Latitude'], df_merged1['Longitude'], df_merged1['Borough'],df_merged1['Neighbourhood']):
    label='{},{}'.format('Neighbourhood','Borough') 
    label=folium.Popup(label,parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto1)  
    
map_toronto1

In [115]:
from sklearn.cluster import KMeans
kclusters=5
toronto_clusters=t_grouped.drop('Neighbourhood',1)
kmeans=KMeans(n_clusters=kclusters,random_state=0).fit(toronto_clusters)
kmeans.labels_

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 4, 1, 1, 1, 4, 1, 1, 1, 3, 1, 1, 4, 1, 1, 1, 4,
       2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1,
       4, 1, 4, 1, 1, 1, 1, 4], dtype=int32)

In [126]:
neighbourhoods_venues_sorted.insert(0,'Cluster Labels', kmeans.labels_)

In [127]:
df_merged2=df_merged
df_merged2=df_merged2.merge(neighbourhoods_venues_sorted, on='Neighbourhood')

In [128]:
df_merged2.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1 Most Common Venue,2 Most Common Venue,3 Most Common Venue,4 Most Common Venue,5 Most Common Venue,6 Most Common Venue,7 Most Common Venue,8 Most Common Venue,9 Most Common Venue,10 Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,4,Park,Food & Drink Shop,Yoga Studio,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
1,M4A,North York,Victoria Village,43.725882,-79.315572,1,Portuguese Restaurant,Coffee Shop,Hockey Arena,French Restaurant,Intersection,Pizza Place,Drugstore,Donut Shop,Doner Restaurant,Dog Run
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1,Coffee Shop,Pub,Bakery,Park,Theater,Breakfast Spot,Café,Event Space,Restaurant,Performing Arts Venue
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,1,Furniture / Home Store,Clothing Store,Accessories Store,Athletics & Sports,Event Space,Miscellaneous Shop,Coffee Shop,Boutique,Women's Store,Vietnamese Restaurant
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1,Coffee Shop,Diner,Yoga Studio,College Cafeteria,Bar,Beer Bar,Smoothie Shop,Sandwich Place,Burrito Place,Café


In [129]:
# create map
map_clusters = folium.Map(location = [43.6532,-79.3832], zoom_start = 11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i * x) ** 2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_merged2['Latitude'], df_merged2['Longitude'], df_merged2['Neighbourhood'], df_merged2['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius = 5,
        popup = label,
        color = rainbow[cluster - 1],
        fill = True,
        fill_color = rainbow[cluster - 1],
        fill_opacity = 0.7).add_to(map_clusters)
       
map_clusters

In [135]:
#Examine each cluster
df_merged2.loc[df_merged2['Cluster Labels'] == 0, df_merged2.columns[[2] + list(range(5, df_merged2.shape[1]))]]

,Neighbourhood,Cluster Labels,1 Most Common Venue,2 Most Common Venue,3 Most Common Venue,4 Most Common Venue,5 Most Common Venue,6 Most Common Venue,7 Most Common Venue,8 Most Common Venue,9 Most Common Venue,10 Most Common Venue
5,"Malvern, Rouge",0,Fast Food Restaurant,Yoga Studio,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run


There is only 1 neighbourhood in cluster 1, and it has fast food as its most common venue

In [144]:
#cluster 2
cluster2=df_merged2.loc[df_merged2['Cluster Labels'] == 1, df_merged2.columns[[2] + list(range(5, df_merged2.shape[1]))]]
cluster2
cluster2['Neighbourhood']=

,Neighbourhood,Cluster Labels,1 Most Common Venue,2 Most Common Venue,3 Most Common Venue,4 Most Common Venue,5 Most Common Venue,6 Most Common Venue,7 Most Common Venue,8 Most Common Venue,9 Most Common Venue,10 Most Common Venue
1,Victoria Village,1,Portuguese Restaurant,Coffee Shop,Hockey Arena,French Restaurant,Intersection,Pizza Place,Drugstore,Donut Shop,Doner Restaurant,Dog Run
2,"Regent Park, Harbourfront",1,Coffee Shop,Pub,Bakery,Park,Theater,Breakfast Spot,Café,Event Space,Restaurant,Performing Arts Venue
3,"Lawrence Manor, Lawrence Heights",1,Furniture / Home Store,Clothing Store,Accessories Store,Athletics & Sports,Event Space,Miscellaneous Shop,Coffee Shop,Boutique,Women's Store,Vietnamese Restaurant
4,"Queen's Park, Ontario Provincial Government",1,Coffee Shop,Diner,Yoga Studio,College Cafeteria,Bar,Beer Bar,Smoothie Shop,Sandwich Place,Burrito Place,Café
6,Don Mills,1,Gym,Coffee Shop,Beer Store,Restaurant,Japanese Restaurant,Art Gallery,Chinese Restaurant,Sporting Goods Shop,Asian Restaurant,Clothing Store
...,...,...,...,...,...,...,...,...,...,...,...,...
93,"St. James Town, Cabbagetown",1,Pizza Place,Coffee Shop,Bakery,Pub,Park,Italian Restaurant,Chinese Restaurant,Restaurant,Café,Butcher
94,"First Canadian Place, Underground city",1,Coffee Shop,Café,Hotel,Restaurant,Gym,Japanese Restaurant,Seafood Restaurant,Asian Restaurant,Steakhouse,American Restaurant
96,Church and Wellesley,1,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Gay Bar,Restaurant,Hotel,Bubble Tea Shop,Mediterranean Restaurant,Yoga Studio,Men's Store
97,"Business reply mail Processing Centre, South C...",1,Yoga Studio,Garden,Light Rail Station,Burrito Place,Farmers Market,Fast Food Restaurant,Butcher,Restaurant,Recording Studio,Brewery


There are 86 neighbourhoods in cluster 2

In [142]:
cluster3=df_merged2.loc[df_merged2['Cluster Labels'] == 2, df_merged2.columns[[2] + list(range(5, df_merged2.shape[1]))]]
cluster3


,Neighbourhood,Cluster Labels,1 Most Common Venue,2 Most Common Venue,3 Most Common Venue,4 Most Common Venue,5 Most Common Venue,6 Most Common Venue,7 Most Common Venue,8 Most Common Venue,9 Most Common Venue,10 Most Common Venue
60,Roselawn,2,Garden,Yoga Studio,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Dance Studio


Only 1 neighbourhood in cluster 3

In [138]:
df_merged2.loc[df_merged2['Cluster Labels'] == 3, df_merged2.columns[[2] + list(range(5, df_merged2.shape[1]))]]


,Neighbourhood,Cluster Labels,1 Most Common Venue,2 Most Common Venue,3 Most Common Venue,4 Most Common Venue,5 Most Common Venue,6 Most Common Venue,7 Most Common Venue,8 Most Common Venue,9 Most Common Venue,10 Most Common Venue
56,"Humberlea, Emery",3,Baseball Field,Yoga Studio,Department Store,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant
98,"Old Mill South, King's Mill Park, Sunnylea, Hu...",3,Baseball Field,Yoga Studio,Department Store,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant


2 neighbourhoods in cluster 4, both with baseball field as the most common place.
Looking at this cluster its very clear to see that the neighbourhoods are very comparable in terms of popular venues, with all 10 top venues matching for both neighbourhoods


In [139]:
df_merged2.loc[df_merged2['Cluster Labels'] == 4, df_merged2.columns[[2] + list(range(5, df_merged2.shape[1]))]]


,Neighbourhood,Cluster Labels,1 Most Common Venue,2 Most Common Venue,3 Most Common Venue,4 Most Common Venue,5 Most Common Venue,6 Most Common Venue,7 Most Common Venue,8 Most Common Venue,9 Most Common Venue,10 Most Common Venue
0,Parkwoods,4,Park,Food & Drink Shop,Yoga Studio,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
20,Caledonia-Fairbanks,4,Park,Women's Store,Pool,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doner Restaurant
34,"East Toronto, Broadview North (Old East York)",4,Convenience Store,Park,Coffee Shop,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run
49,"North Park, Maple Leaf Park, Upwood Park",4,Park,Bakery,Construction & Landscaping,Yoga Studio,Dog Run,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Donut Shop
52,"Willowdale, Newtonbrook",4,Park,Yoga Studio,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doner Restaurant
62,Weston,4,Convenience Store,Park,Yoga Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
64,York Mills West,4,Convenience Store,Park,Yoga Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
83,"Milliken, Agincourt North, Steeles East, L'Amo...",4,Park,Playground,Yoga Studio,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doner Restaurant
89,Rosedale,4,Park,Trail,Playground,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
95,"The Kingsway, Montgomery Road, Old Mill North",4,Park,Pool,River,College Stadium,Colombian Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run


There are 10 neighbourhoods in cluster 5, with most of the top 10 venues matching but not necessarily in the same order